In [1]:
import numpy as np
from tensorflow.python.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.initializers import VarianceScaling
from sklearn.cluster import KMeans
from custom_layers import autoencoder as auto_encoder
from custom_layers import ClusteringLayer
import cv2
import os, glob, shutil

In [2]:
# computing an auxiliary target distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

In [3]:
x_train = [cv2.resize(cv2.imread(file), (224, 224)) for file in glob.glob("input/train/images/*.png")]
x_test = [cv2.resize(cv2.imread(file), (224, 224)) for file in glob.glob("input/test/images/*.png")]
print("Images loaded")

Images loaded


In [4]:
n_clusters = 5  # No. of clusters
x = np.concatenate((x_train,x_test))
x = x.reshape((x.shape[0], -1))
x = np.divide(x, 255.)

In [5]:
# dim[0] = og dimensions (w * h) dim[1:] = layers of auto encoder. 
# Numbers equal the shape the encoder forces the data into in the layer.
dims = [x.shape[-1], 500, 500, 2000, 10]
init = VarianceScaling(scale=1. / 3., mode='fan_in', distribution='uniform')
pretrain_optimizer = SGD(learning_rate=1, momentum=0.9)
pretrain_epochs = 50
batch_size = 500
save_dir = './weights'

In [6]:
autoencoder, encoder = auto_encoder(dims, init=init)
autoencoder.compile(optimizer=pretrain_optimizer, loss='mse')
print("Starting training")
autoencoder.fit(x, x, batch_size=batch_size, epochs=pretrain_epochs)
autoencoder.save_weights(save_dir + '/ae_weights.h5')

Starting training
Epoch 1/30
4/4 [==============================] - 57s 10s/step - loss: 0.2539
Epoch 2/30
4/4 [==============================] - 41s 10s/step - loss: 0.2538
Epoch 3/30
4/4 [==============================] - 38s 9s/step - loss: 0.2536
Epoch 4/30
4/4 [==============================] - 22s 5s/step - loss: 0.2535
Epoch 5/30
4/4 [==============================] - 22s 5s/step - loss: 0.2532
Epoch 6/30
4/4 [==============================] - 29s 8s/step - loss: 0.2530
Epoch 7/30
4/4 [==============================] - 25s 6s/step - loss: 0.2528
Epoch 8/30
4/4 [==============================] - 24s 6s/step - loss: 0.2525
Epoch 9/30
4/4 [==============================] - 22s 6s/step - loss: 0.2522
Epoch 10/30
4/4 [==============================] - 22s 5s/step - loss: 0.2520
Epoch 11/30
4/4 [==============================] - 23s 5s/step - loss: 0.2517
Epoch 12/30
4/4 [==============================] - 24s 6s/step - loss: 0.2513
Epoch 13/30
4/4 [==============================] - 23

In [7]:
autoencoder.save_weights(save_dir + '/ae_weights.h5')

In [8]:
autoencoder.load_weights(save_dir + '/ae_weights.h5')

In [9]:
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=clustering_layer)
model.compile(optimizer=SGD(0.01, 0.9), loss='kld')

In [10]:
kmeans = KMeans(n_clusters=n_clusters, n_init=100, verbose=True, max_iter = 1000)
y_pred = kmeans.fit_predict(encoder.predict(x))
y_pred_last = np.copy(y_pred)
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 131.71251
start iteration
done sorting
end inner loop
Iteration 1, inertia 115.05906
start iteration
done sorting
end inner loop
Iteration 2, inertia 111.20394
start iteration
done sorting
end inner loop
Iteration 3, inertia 108.28248
start iteration
done sorting
end inner loop
Iteration 4, inertia 107.13602
start iteration
done sorting
end inner loop
Iteration 5, inertia 106.60761
start iteration
done sorting
end inner loop
Iteration 6, inertia 106.00728
start iteration
done sorting
end inner loop
Iteration 7, inertia 105.58117
start iteration
done sorting
end inner loop
Iteration 8, inertia 105.07763
start iteration
done sorting
end inner loop
Iteration 9, inertia 104.546
start iteration
done sorting
end inner loop
Iteration 10, inertia 104.30462
start iteration
done sorting
end inner loop
Iteration 11, inertia 104.11449
start iteration
done sorting
end inner loop
Iteration 12, inertia 104.01971


Iteration 10, inertia 115.89616
start iteration
done sorting
end inner loop
Iteration 11, inertia 112.86659
start iteration
done sorting
end inner loop
Iteration 12, inertia 111.13982
start iteration
done sorting
end inner loop
Iteration 13, inertia 110.26664
start iteration
done sorting
end inner loop
Iteration 14, inertia 109.04454
start iteration
done sorting
end inner loop
Iteration 15, inertia 108.060555
start iteration
done sorting
end inner loop
Iteration 16, inertia 107.4953
start iteration
done sorting
end inner loop
Iteration 17, inertia 107.19612
start iteration
done sorting
end inner loop
Iteration 18, inertia 106.79816
start iteration
done sorting
end inner loop
Iteration 19, inertia 106.45722
start iteration
done sorting
end inner loop
Iteration 20, inertia 106.03467
start iteration
done sorting
end inner loop
Iteration 21, inertia 105.54957
start iteration
done sorting
end inner loop
Iteration 22, inertia 105.188805
start iteration
done sorting
end inner loop
Iteration 2

Iteration 16, inertia 109.52537
start iteration
done sorting
end inner loop
Iteration 17, inertia 108.609924
start iteration
done sorting
end inner loop
Iteration 18, inertia 107.74815
start iteration
done sorting
end inner loop
Iteration 19, inertia 107.37694
start iteration
done sorting
end inner loop
Iteration 20, inertia 107.050415
start iteration
done sorting
end inner loop
Iteration 21, inertia 106.683075
start iteration
done sorting
end inner loop
Iteration 22, inertia 106.34646
start iteration
done sorting
end inner loop
Iteration 23, inertia 105.82929
start iteration
done sorting
end inner loop
Iteration 24, inertia 105.45047
start iteration
done sorting
end inner loop
Iteration 25, inertia 104.98572
start iteration
done sorting
end inner loop
Iteration 26, inertia 104.56405
start iteration
done sorting
end inner loop
Iteration 27, inertia 104.24219
start iteration
done sorting
end inner loop
Iteration 28, inertia 103.84205
start iteration
done sorting
end inner loop
Iteration

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 105.05332
start iteration
done sorting
end inner loop
Iteration 1, inertia 103.79309
start iteration
done sorting
end inner loop
Iteration 2, inertia 103.61464
start iteration
done sorting
end inner loop
Iteration 3, inertia 103.574234
start iteration
done sorting
end inner loop
Iteration 4, inertia 103.5635
center shift 4.825250e-06 within tolerance 1.052177e-05
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 148.61668
start iteration
done sorting
end inner loop
Iteration 1, inertia 126.79523
start iteration
done sorting
end inner loop
Iteration 2, inertia 118.10009
start iteration
done sorting
end inner loop
Iteration 3, inertia 114.302414
start iteration
done sorting
end inner loop
Iteration 4, inertia 111.49837
start iteration
done sorting
end inner loop
Iteration 5, inertia 108.70247
start iteration
done sorting
end inner loop
Iteration 6, inertia 107.2

Iteration 0, inertia 111.5342
start iteration
done sorting
end inner loop
Iteration 1, inertia 108.33475
start iteration
done sorting
end inner loop
Iteration 2, inertia 106.52
start iteration
done sorting
end inner loop
Iteration 3, inertia 105.62713
start iteration
done sorting
end inner loop
Iteration 4, inertia 105.22094
start iteration
done sorting
end inner loop
Iteration 5, inertia 104.67769
start iteration
done sorting
end inner loop
Iteration 6, inertia 104.386154
start iteration
done sorting
end inner loop
Iteration 7, inertia 104.15582
start iteration
done sorting
end inner loop
Iteration 8, inertia 104.040825
start iteration
done sorting
end inner loop
Iteration 9, inertia 103.95805
start iteration
done sorting
end inner loop
Iteration 10, inertia 103.8576
start iteration
done sorting
end inner loop
Iteration 11, inertia 103.74772
start iteration
done sorting
end inner loop
Iteration 12, inertia 103.67476
start iteration
done sorting
end inner loop
Iteration 13, inertia 103

...

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 108.86795
start iteration
done sorting
end inner loop
Iteration 1, inertia 106.04904
start iteration
done sorting
end inner loop
Iteration 2, inertia 105.150665
start iteration
done sorting
end inner loop
Iteration 3, inertia 104.657814
start iteration
done sorting
end inner loop
Iteration 4, inertia 104.44471
start iteration
done sorting
end inner loop
Iteration 5, inertia 104.10152
start iteration
done sorting
end inner loop
Iteration 6, inertia 103.67808
start iteration
done sorting
end inner loop
Iteration 7, inertia 103.570656
start iteration
done sorting
end inner loop
Iteration 8, inertia 103.56587
center shift 9.743787e-06 within tolerance 1.052177e-05
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 126.50963
start iteration
done sorting
end inner loop
Iteration 1, inertia 125.51009
start iteration
done sorting
end inner loop
Iteration 2, inertia 124

Iteration 2, inertia 107.91412
start iteration
done sorting
end inner loop
Iteration 3, inertia 106.80414
start iteration
done sorting
end inner loop
Iteration 4, inertia 106.13293
start iteration
done sorting
end inner loop
Iteration 5, inertia 105.51023
start iteration
done sorting
end inner loop
Iteration 6, inertia 104.7283
start iteration
done sorting
end inner loop
Iteration 7, inertia 104.48066
start iteration
done sorting
end inner loop
Iteration 8, inertia 104.15729
start iteration
done sorting
end inner loop
Iteration 9, inertia 103.72226
start iteration
done sorting
end inner loop
Iteration 10, inertia 103.570656
start iteration
done sorting
end inner loop
Iteration 11, inertia 103.56587
center shift 9.743787e-06 within tolerance 1.052177e-05
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 151.35718
start iteration
done sorting
end inner loop
Iteration 1, inertia 132.5339
start iteration
done sorting
end inner loop
Iteration 2, inerti

Iteration 18, inertia 103.615204
start iteration
done sorting
end inner loop
Iteration 19, inertia 103.58315
start iteration
done sorting
end inner loop
Iteration 20, inertia 103.5652
start iteration
done sorting
end inner loop
Iteration 21, inertia 103.5635
center shift 1.110693e-06 within tolerance 1.052177e-05
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 136.99527
start iteration
done sorting
end inner loop
Iteration 1, inertia 122.606705
start iteration
done sorting
end inner loop
Iteration 2, inertia 115.26992
start iteration
done sorting
end inner loop
Iteration 3, inertia 111.183685
start iteration
done sorting
end inner loop
Iteration 4, inertia 109.95449
start iteration
done sorting
end inner loop
Iteration 5, inertia 108.87518
start iteration
done sorting
end inner loop
Iteration 6, inertia 108.0153
start iteration
done sorting
end inner loop
Iteration 7, inertia 107.493095
start iteration
done sorting
end inner loop
Iteration 8, in

Iteration 25, inertia 103.56587
start iteration
done sorting
end inner loop
Iteration 26, inertia 103.56587
center shift 0.000000e+00 within tolerance 1.052177e-05
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 112.71149
start iteration
done sorting
end inner loop
Iteration 1, inertia 105.657715
start iteration
done sorting
end inner loop
Iteration 2, inertia 104.48587
start iteration
done sorting
end inner loop
Iteration 3, inertia 104.04981
start iteration
done sorting
end inner loop
Iteration 4, inertia 103.845856
start iteration
done sorting
end inner loop
Iteration 5, inertia 103.69291
start iteration
done sorting
end inner loop
Iteration 6, inertia 103.61242
start iteration
done sorting
end inner loop
Iteration 7, inertia 103.57794
start iteration
done sorting
end inner loop
Iteration 8, inertia 103.5652
center shift 9.925176e-06 within tolerance 1.052177e-05
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0,

Iteration 6, inertia 103.952736
start iteration
done sorting
end inner loop
Iteration 7, inertia 103.75754
start iteration
done sorting
end inner loop
Iteration 8, inertia 103.59371
start iteration
done sorting
end inner loop
Iteration 9, inertia 103.56587
center shift 9.895273e-06 within tolerance 1.052177e-05
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 125.337204
start iteration
done sorting
end inner loop
Iteration 1, inertia 118.68942
start iteration
done sorting
end inner loop
Iteration 2, inertia 114.524124
start iteration
done sorting
end inner loop
Iteration 3, inertia 111.88144
start iteration
done sorting
end inner loop
Iteration 4, inertia 110.843346
start iteration
done sorting
end inner loop
Iteration 5, inertia 110.007774
start iteration
done sorting
end inner loop
Iteration 6, inertia 108.949295
start iteration
done sorting
end inner loop
Iteration 7, inertia 108.060555
start iteration
done sorting
end inner loop
Iteration 8, 

In [11]:
model.save_weights(save_dir + '/DEC_model_final.h5')

In [12]:
model.load_weights(save_dir + '/DEC_model_final.h5')

In [13]:
paths = glob.glob("input/train/images/*.png")
print("paths found")
for i in range(n_clusters):
    if os.path.exists("output/cluster" + str(i)):
        shutil.rmtree("output/cluster" + str(i))
        os.makedirs("output/cluster" + str(i))
    else:
        os.makedirs("output/cluster" + str(i))
    print("output/cluster" + str(i))
print("Moving Images")
for i in range(len(paths)):
    shutil.copy2(paths[i], "output/cluster"+str(y_pred_last[i]))
    print(str(i+1) + "/" + str(len(paths)) + " Images Copied")

paths found
output/cluster0
output/cluster1
output/cluster2
output/cluster3
output/cluster4
Moving Images
1/1493 Images Copied
2/1493 Images Copied
3/1493 Images Copied
4/1493 Images Copied
5/1493 Images Copied
6/1493 Images Copied
7/1493 Images Copied
8/1493 Images Copied
9/1493 Images Copied
10/1493 Images Copied
11/1493 Images Copied
12/1493 Images Copied
13/1493 Images Copied
14/1493 Images Copied
15/1493 Images Copied
16/1493 Images Copied
17/1493 Images Copied
18/1493 Images Copied
19/1493 Images Copied
20/1493 Images Copied
21/1493 Images Copied
22/1493 Images Copied
23/1493 Images Copied
24/1493 Images Copied
25/1493 Images Copied
26/1493 Images Copied
27/1493 Images Copied
28/1493 Images Copied
29/1493 Images Copied
30/1493 Images Copied
31/1493 Images Copied
32/1493 Images Copied
33/1493 Images Copied
34/1493 Images Copied
35/1493 Images Copied
36/1493 Images Copied
37/1493 Images Copied
38/1493 Images Copied
39/1493 Images Copied
40/1493 Images Copied
41/1493 Images Copied
4

373/1493 Images Copied
374/1493 Images Copied
375/1493 Images Copied
376/1493 Images Copied
377/1493 Images Copied
378/1493 Images Copied
379/1493 Images Copied
380/1493 Images Copied
381/1493 Images Copied
382/1493 Images Copied
383/1493 Images Copied
384/1493 Images Copied
385/1493 Images Copied
386/1493 Images Copied
387/1493 Images Copied
388/1493 Images Copied
389/1493 Images Copied
390/1493 Images Copied
391/1493 Images Copied
392/1493 Images Copied
393/1493 Images Copied
394/1493 Images Copied
395/1493 Images Copied
396/1493 Images Copied
397/1493 Images Copied
398/1493 Images Copied
399/1493 Images Copied
400/1493 Images Copied
401/1493 Images Copied
402/1493 Images Copied
403/1493 Images Copied
404/1493 Images Copied
405/1493 Images Copied
406/1493 Images Copied
407/1493 Images Copied
408/1493 Images Copied
409/1493 Images Copied
410/1493 Images Copied
411/1493 Images Copied
412/1493 Images Copied
413/1493 Images Copied
414/1493 Images Copied
415/1493 Images Copied
416/1493 Im

752/1493 Images Copied
753/1493 Images Copied
754/1493 Images Copied
755/1493 Images Copied
756/1493 Images Copied
757/1493 Images Copied
758/1493 Images Copied
759/1493 Images Copied
760/1493 Images Copied
761/1493 Images Copied
762/1493 Images Copied
763/1493 Images Copied
764/1493 Images Copied
765/1493 Images Copied
766/1493 Images Copied
767/1493 Images Copied
768/1493 Images Copied
769/1493 Images Copied
770/1493 Images Copied
771/1493 Images Copied
772/1493 Images Copied
773/1493 Images Copied
774/1493 Images Copied
775/1493 Images Copied
776/1493 Images Copied
777/1493 Images Copied
778/1493 Images Copied
779/1493 Images Copied
780/1493 Images Copied
781/1493 Images Copied
782/1493 Images Copied
783/1493 Images Copied
784/1493 Images Copied
785/1493 Images Copied
786/1493 Images Copied
787/1493 Images Copied
788/1493 Images Copied
789/1493 Images Copied
790/1493 Images Copied
791/1493 Images Copied
792/1493 Images Copied
793/1493 Images Copied
794/1493 Images Copied
795/1493 Im

1104/1493 Images Copied
1105/1493 Images Copied
1106/1493 Images Copied
1107/1493 Images Copied
1108/1493 Images Copied
1109/1493 Images Copied
1110/1493 Images Copied
1111/1493 Images Copied
1112/1493 Images Copied
1113/1493 Images Copied
1114/1493 Images Copied
1115/1493 Images Copied
1116/1493 Images Copied
1117/1493 Images Copied
1118/1493 Images Copied
1119/1493 Images Copied
1120/1493 Images Copied
1121/1493 Images Copied
1122/1493 Images Copied
1123/1493 Images Copied
1124/1493 Images Copied
1125/1493 Images Copied
1126/1493 Images Copied
1127/1493 Images Copied
1128/1493 Images Copied
1129/1493 Images Copied
1130/1493 Images Copied
1131/1493 Images Copied
1132/1493 Images Copied
1133/1493 Images Copied
1134/1493 Images Copied
1135/1493 Images Copied
1136/1493 Images Copied
1137/1493 Images Copied
1138/1493 Images Copied
1139/1493 Images Copied
1140/1493 Images Copied
1141/1493 Images Copied
1142/1493 Images Copied
1143/1493 Images Copied
1144/1493 Images Copied
1145/1493 Images

1462/1493 Images Copied
1463/1493 Images Copied
1464/1493 Images Copied
1465/1493 Images Copied
1466/1493 Images Copied
1467/1493 Images Copied
1468/1493 Images Copied
1469/1493 Images Copied
1470/1493 Images Copied
1471/1493 Images Copied
1472/1493 Images Copied
1473/1493 Images Copied
1474/1493 Images Copied
1475/1493 Images Copied
1476/1493 Images Copied
1477/1493 Images Copied
1478/1493 Images Copied
1479/1493 Images Copied
1480/1493 Images Copied
1481/1493 Images Copied
1482/1493 Images Copied
1483/1493 Images Copied
1484/1493 Images Copied
1485/1493 Images Copied
1486/1493 Images Copied
1487/1493 Images Copied
1488/1493 Images Copied
1489/1493 Images Copied
1490/1493 Images Copied
1491/1493 Images Copied
1492/1493 Images Copied
1493/1493 Images Copied


In [14]:
for 
#x_train = [cv2.resize(cv2.imread(file), (224, 224)) for file in glob.glob("output/cluster0/*.png")]

SyntaxError: invalid syntax (<ipython-input-14-31cbbd22813d>, line 1)